In [ ]:
import json
from tools.exam.exam_validator import ExamValidator

# 시험 데이터 로드
with open('evaluation/eval_data/4_multiple_exam/계산문제_exam.json', 'r', encoding='utf-8') as f:
    exam_data = json.load(f)

# 검증 수행
result = ExamValidator.validate_multiple_choice_format(exam_data)

# 결과 출력
report = ExamValidator.print_validation_report(result)
print(report)

In [ ]:
from tools.data_processing import JSONCleaner

cleaner = JSONCleaner(verbose=True)

# 디렉토리 전체 정리 + 리포트 생성
result = cleaner.cleanup_directory(
    "data/FINAL/3C",
    generate_report=True
)